<h2>Zadání č 8.
Analýza plánovaných hodin na učebně v daném časovém období (gql_facilities + gql_events)</h2>

- Vytvořit GQL dotaz na základě existující federace,
- Definovat transformaci GQL response -> table rows (vstup pro kontingenční tabulku)
- Vytvořit kontingenční tabulku
- Vytvořit koláčový / sloupcový graf
- Vytvořit Sunburst / Chord graf
- Výsledek realizujte jako ipynb notebook (autentizace jménem a heslem, realizace aiohttp, transformace response, vytvoření tabulky, vytvoření grafu).

<b>Instalace potřebných knihoven</b>

In [4]:
%pip install asyncio nbformat aiohttp plotly.express pandas 


Note: you may need to restart the kernel to use updated packages.


In [5]:
import json
import aiohttp
import pandas as pd
import plotly.express as px
from itertools import product
from functools import reduce


<b>Funkce pro získání tokenu</b>

In [6]:
async def getToken(username, password):
    keyurl = "http://localhost:33001/oauth/login3"
    async with aiohttp.ClientSession() as session:
        async with session.get(keyurl) as resp:
            keyJson = await resp.json()

        payload = {"key": keyJson["key"], "username": username, "password": password}
        async with session.post(keyurl, json=payload) as resp:
            tokenJson = await resp.json()
    return tokenJson.get("token", None)

<b>Funkce pro definici GQL dotazu</b>

In [7]:
def query(q, token):
    async def post(variables):
        gqlurl = "http://localhost:33001/api/gql"
        payload = {"query": q, "variables": variables}
        cookies = {'authorization': token}
        async with aiohttp.ClientSession() as session:
            async with session.post(gqlurl, json=payload, cookies=cookies) as resp:
                if resp.status != 200:
                    text = await resp.text()
                    print(text)
                    return text
                else:
                    response = await resp.json()
                    return response
    return post

<b>Přihlašovací údaje</b>

In [8]:
username = "john.newbie@world.com"
password = "john.newbie@world.com"

<b>GQL dotaz</b>

In [9]:
queryStr = """
{
  eventPage {
    id
    name
    placeId
    place
    startdate
    enddate
    eventType{
      id
      name
    }
    groups{
      id
      name
    }
  }
}
"""

<b>Mapování</b>

In [10]:
mappers = {
    "eventID": "id",
    "eventName": "name",
    "placeID": "placeId",
    "place" :"place",
    "startDate": "startdate",
    "endDate": "enddate",
    "eventTypeID": "eventType.id",
    "eventType": "eventType.name",
    "groupID": "groups.id",
    "groupName": "groups.name",
}

<b>Tabulka</b>

In [11]:
def createTable(tableData):
    pd.set_option('display.max_columns', None)
    pd.set_option('display.expand_frame_repr', False)
    df = pd.DataFrame(tableData)

    print(df)

<b>Grafy</b>

In [19]:
def createGraphs(graphData):
    df = pd.DataFrame(graphData)

    fig_sunburst = px.sunburst(
        df,
        path=['groupName','place', 'eventName'], 
        title='Event Distribution by Group, Place and Event'
    )

    fig_sunburst.update_layout(
        width=800,  
        height=800 
    )

    df_bar = df.groupby(['eventName']).size().reset_index(name='counts')

    fig_bar = px.bar(
        df_bar,
        x='eventName',
        y='counts',
        barmode='group',
        title='Number of Events by Event Name',
        labels={'eventName': 'Event Name', 'counts': 'Number of Events'}
    )

    fig_bar.update_layout(
        yaxis=dict(
            tickmode='linear',
            dtick=1
        )
    )

    fig_sunburst.show()
    fig_bar.show()

<b>Pomocné funkce</b>

In [13]:
def enumerateAttrs(attrs):
    for key, value in attrs.items():
        names = value.split(".")
        name = names[0]
        yield key, name

def flattenList(inList, outItem, attrs):
    for item in inList:
        assert isinstance(item, dict), f"in list only dicts are expected"
        for row in flatten(item, outItem, attrs):
            yield row

def flattenDict(inDict, outItem, attrs):
    result = {**outItem}
    complexAttrs = []
    for key, value in enumerateAttrs(attrs):
        attributeValue = inDict.get(value, None)
        if isinstance(attributeValue, list):
            complexAttrs.append((key, value))
        elif isinstance(attributeValue, dict):
            complexAttrs.append((key, value))
        else:
            result[key] = attributeValue
    lists = []
    for key, value in complexAttrs:
        attributeValue = inDict.get(value, None)
        prefix = f"{value}."
        prefixlen = len(prefix)
        subAttrs = {key: value[prefixlen:] for key, value in attrs.items() if value.startswith(prefix)}
        items = list(flatten(attributeValue, result, subAttrs))
        lists.append(items)
                     
    if len(lists) == 0:
        yield result
    else:
        for element in product(*lists):
            reduced = reduce(lambda a, b: {**a, **b}, element, {})
            yield reduced

def flatten(inData, outItem, attrs):
    if isinstance(inData, dict):
        for item in flattenDict(inData, outItem, attrs):
            yield item
    elif isinstance(inData, list):
        for item in flattenList(inData, outItem, attrs):
            yield item
    else:
        assert False, f"Unexpected type on inData {inData}"

<b>Asynchronní funkce pro proces</b>

In [69]:
async def fullPipe():
    token = await getToken(username, password)
    qfunc = query(queryStr, token)
    response = await qfunc({})

    data = response.get("data", None)
    result = data.get("eventPage", None)
    
    resultMapped = list(map(lambda event: {**event, "groupsCount": len(event.get("group", []))}, result))
    flatData = list(flatten(resultMapped, {}, mappers))
    
    createTable(flatData)
    createGraphs(flatData)

    return flatData

async def main():
    pipeResult = await fullPipe()
    with open('resultNotebook.json', "w", encoding='utf-8') as outputFile:
        json.dump(pipeResult, outputFile)

await main()

                                eventID   eventName placeID place            startDate              endDate groupID groupName                           eventTypeID      eventType
0  4dccf52f-4117-403c-932a-5691c0d020b1     2021/22    None  None  2021-09-01T00:00:00  2022-09-01T00:00:00    None      None  a517c2fd-8dc7-4a2e-a107-cbdb88ba2aa5     Školní rok
1  5194663f-11aa-4775-91ed-5f3d79269fed     2022/23    None  None  2022-09-01T00:00:00  2023-09-01T00:00:00    None      None  a517c2fd-8dc7-4a2e-a107-cbdb88ba2aa5     Školní rok
2  a64871f8-2308-48ff-adb2-33fb0b0741f1     2023/24    None  None  2023-09-01T00:00:00  2024-09-01T00:00:00    None      None  a517c2fd-8dc7-4a2e-a107-cbdb88ba2aa5     Školní rok
3  9d05f000-7a9d-495d-9257-b597c5c6cb6a     2024/25    None  None  2024-09-01T00:00:00  2025-09-01T00:00:00    None      None  a517c2fd-8dc7-4a2e-a107-cbdb88ba2aa5     Školní rok
4  08ff1c5d-9891-41f6-a824-fc6272adc189  2022/23 ZS    None  None  2022-09-01T00:00:00  2023-03-01T00:00:

ValueError: ('None entries cannot have not-None children', groupName       None
place           None
eventName    2021/22
Name: 0, dtype: object)

<h2>FAKE DATA</h2>

<b>Zpracování falešných dat</b>

In [14]:
def createTableRow(event):
    rows = []
    for group in event["groups"]:
        row = {}
        row["eventName"] = event["name"]
        row["place"] = event["place"]
        row["startDate"] = event["startdate"]
        row["endDate"] = event["enddate"]
        row["eventType"] = event["eventType"]["name"]
        row["groupID"] = group["id"]
        row["groupName"] = group["name"]
        rows.append(row)
    return rows

with open('hard_data.json', encoding='utf-8') as inputFile:
    data = json.load(inputFile)

sourceTable = []


for event in data["data"]["eventPage"]:
    rows = createTableRow(event)
    sourceTable.extend(rows)

with open('result_hard_data.json', "w", encoding='utf-8') as outputFile:
    json.dump(sourceTable, outputFile)

<b>Tabulka</b>

In [15]:
createTable(sourceTable)

                              eventName    place            startDate              endDate  eventType                               groupID groupName
0   Aplikované bezpečnostní technologie   Š3/307  2024-05-27T08:00:00  2024-05-27T09:30:00  Laboratoř  9baf3b54-ae0f-11ed-9bd8-0242ac110002  22-5KB-A
1                        Anglický jazyk  K44/266  2024-05-27T09:50:00  2024-05-27T11:20:00    Cvičení  9baf3b54-ae0f-11ed-9bd8-0242ac110002  22-5KB-A
2            Analýzy a kritické myšlení     T118  2024-05-27T11:40:00  2024-05-27T13:10:00    Cvičení  9baf3b54-ae0f-11ed-9bd8-0242ac110002  22-5KB-A
3                                   VOP  K44/109  2024-05-27T14:30:00  2024-05-27T16:00:00  Přednáška  9baf3b54-ae0f-11ed-9bd8-0242ac110002  22-5KB-A
4               Kybernetická bezpečnost   Š3/307  2024-05-28T08:00:00  2024-05-28T09:30:00    Cvičení  9baf3b54-ae0f-11ed-9bd8-0242ac110002  22-5KB-A
5           Analýza informačních zdrojů  Š9A/67a  2024-05-28T09:50:00  2024-05-28T11:20:00  Laborato

<b>Grafy</b>

In [20]:
with open("result_hard_data.json", "r") as file:
    data = json.load(file)

createGraphs(data)